## Trabajo con reporte de datos de ETABS o SAP2000, ...

Solicitar informacion a ETABS o SAP200 para filtrar y hacer reporte personalizado de `Datos`

### 1. Dependecnias

In [1]:
import numpy as np, pandas as pd
import os, sys
# from matplotlib import pyplot as plt
import comtypes.client
import ipywidgets as widgets

<!-- ### 2. Importar la funcion o metodo para conectarnos y poder solicitar informacion -->
### 2. Seleccion de la apliacion

In [2]:
# from CSiAPI.ConnectCSiAPI import metodo
# Indicar el programa a usar
connect_to = 2; #1 para SAP2000 y 2 para ETABS

connect_to_app, smodel = None, None

In [ ]:
conn = {};
if connect_to == 1:  # SAP2000
    name_app = "SAP2000"
    conn = {
        'app_ruta': "C:\\Program Files\\Computers and Structures\\SAP2000 23\\SAP2000.exe",
        'app_adjunto': "CSI.SAP2000.API.SapObject",
        'app_helper': 'SAP2000v1.Helper'
        }
elif (connect_to == 2): # ETABS
    name_app = "ETABS"
    conn = {
        'app_ruta': "C:\\Program Files\\Computers and Structures\\ETABS 21\\ETABS.exe",
        'app_adjunto': "CSI.ETABS.API.ETABSObject",
        'app_helper': "ETABSv1.Helper"
        }
else: 
    print("No tenemos soporte aun")
    sys.exit()

In [ ]:
# match connect_to:
#     case 1:
#         name_app = "SAP2000"
#         conn = {
#             'app_ruta': "C:\\Program Files\\Computers and Structures\\SAP2000 23\\SAP2000.exe",
#             'app_adjunto': "CSI.SAP2000.API.SapObject",
#             'app_helper': 'SAP2000v1.Helper'
#             }
#     case 2:
#         name_app = "ETABS"
#         conn = {
#             'app_ruta': "C:\\Program Files\\Computers and Structures\\ETABS 21\\ETABS.exe",
#             'app_adjunto': "CSI.ETABS.API.ETABSObject",
#             'app_helper': "ETABSv1.Helper"
#             }

### 3. Coneccion a la aplicacion

**Primera forma**

In [ ]:
try:
    connect_to_app = comtypes.client.GetActiveObject(conn["app_adjunto"])
    print(f"Instancia establecida (adjuntada) a {name_app}")
except (OSError, comtypes.COMError):
    print("No running instance of the program found or failed to attach.")
    sys.exit(-1)

**Segunda forma**

In [ ]:
# helper = comtypes.client.CreateObject(conn["app_helper"])
# if connect_to == 1: 
#     helper = helper.QueryInterface(comtypes.gen.SAP2000v1.cHelper)
# else: 
#     helper = helper.QueryInterface(comtypes.gen.ETABSv1.cHelper);
# try:
#     connect_to_app = helper.GetObject(conn["app_adjunto"])
#     print(f"Instancia establecida (adjuntada) a {name_app}")
# except (OSError, comtypes.COMError):
#     print("No running instance of the program found or failed to attach.")
#     sys.exit(-1)

### 4. Se crea el objeto SapModel o instancia a la Clase

In [ ]:
smodel = connect_to_app.SapModel;

### 5. Se establece desbloqueo para ejecutar analisis.

In [ ]:
print(smodel.GetModelIsLocked()) # False

smodel.SetModelIsLocked(False)

### 6. Ejecucion del modelo

In [ ]:
smodel.Analyze.RunAnalysis()

### 7. Solicitud de informacion

In [11]:
# def database(filtro):
#     pass

In [2]:
# radio_btn = widgets.RadioButtons(
#     options=[('MODEL DEFINITION', 1), ('ANALYSIS RESULTS', 2), ('DESIGN DATA', 3)],
#     value=1,
#     description='Filtro:',
#     disabled=False
# )
# widgets.interact(
#     database,
#     filtro=radio_btn, # Parmetro que se le envia
#     value=2,
#     # description='Filtro:'
# )

In [1]:
# accordion = widgets.Accordion(
#     children=[
#         widgets.IntSlider(), 
#         widgets.Text(),
#         widgets.Text()
#     ]
# )
# accordion.set_title(0, 'Slider')
# accordion.set_title(1, 'Text')
# accordion.set_title(2, 'otro')
# # accordion

# tab_nest = widgets.Tab()
# tab_nest.children = [accordion]
# tab_nest.set_title(0, 'An accordion')
# # tab_nest.set_title(1, 'Copy of the accordion')
# tab_nest

In [ ]:
name_tbl = "Modal Participating Mass Ratios"
data = smodel.DatabaseTables.GetTableForDisplayArray(name_tbl, GroupName="")

cols, no_of_rows = data[2], data[3]
data_values = np.array_split(data[4], no_of_rows)
df = pd.DataFrame(data_values)
df.columns = cols
df

### Filtrado y tratamiento de datos

In [ ]:
# df = df.drop(columns=["SumUZ"])
# df

In [ ]:
df = df.drop(['UZ','SumUZ'], axis=1)
df

### Cerrar la coneccion y aplicacion

In [ ]:
if (connect_to_app.ApplicationExit(False)): print("No se pudo cerrar")
else: print("Se cerro con exito.")

smodel, connect_to_app = None, None
del smodel, connect_to_app
exit(-1)

## referencias.

- [ipywidgets](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html)